# Code draft for tweet extraction for fake news detection project 
By: Hedi Fendri 

## Imports 

In [ ]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import preprocessor as p
import numpy as np
import nltk

## Getting Twitter API credntials: Please add you cridentials

In [2]:
#Twitter credentials for the app

# authorization tokens

with open("twitter_credentials_empty.json", "r") as file:
    creds = json.load(file)

consumer_key = creds["CONSUMER_KEY"]
consumer_secret = creds["CONSUMER_SECRET"]
access_key = creds["ACCESS_TOKEN"]
access_secret = creds["ACCESS_SECRET"]


In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [7]:
def get_all_tweets(screen_name):
    """
    This function allows the extraction of the most recent tweets of one user defined by the string
    screen_name
    """

    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)

    #initialize a list to hold all the tweepy Tweets
    alltweets = []

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        #print("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        #print("...%s tweets downloaded so far" % (len(alltweets)))

    #transform the tweepy tweets into a 2D array that will populate the csv	
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]

    #write the csv
    outttweets_df=pd.DataFrame(outtweets)
    outttweets_df.columns=["id","created_at","text"]
    """
    with open('%s_tweets.csv' % screen_name, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text"])
        writer.writerows(outtweets)
    """
    return outttweets_df

    pass

## Getting the most important users for our study from an external csv file

In [8]:
tweeter_accounts=pd.read_csv("twitterDF.csv",delimiter=",")

In [9]:
all_users=np.unique(tweeter_accounts.user.values)

In [10]:
tweeter_accounts.head(5)

Unnamed: 0           user  \
0           0  westland_will   
1           1    nailainayat   
2           2   SaudiProject   
3           3      maggieNYT   
4           4     RexChapman   

                                                text  followers tweet_time  \
0  "Coronavirus" and "COVID-19" are propaganda na...      20463   19:38:34   
1  Gilgit-Baltistan doctor Usama who tested posit...     118585   16:54:12   
2  🔴 عاجل\n\n#الملك_سلمان يصدر أمرًا بمنع التجول ...    1166239   22:52:35   
3  “Romney’s in isolation? Geeeeee - that’s too b...    1259735   22:39:15   
4  I’ve seen some creative and great Coronavirus ...     578631   22:31:06   

   tweet_date account_time account_date       country  
0  2020-03-22     18:57:28   2017-10-18          None  
1  2020-03-22     05:41:18   2011-06-20      Pakistan  
2  2020-03-22     07:56:55   2011-08-01  Saudi Arabia  
3  2020-03-22     23:14:06   2009-11-27          None  
4  2020-03-22     02:36:27   2011-06-27          None

## Let's start scrapping for some predifined users: 
### In average this method allow the extraction of 80 users in a time due to Rate limit error 

In [12]:
tweet_df=pd.DataFrame(columns=["user","created_at","text"])
index=0
keeped_users=all_users[0:100]
for user in keeped_users :
    index=index+1
    comp=(index)/(keeped_users.shape[0])
    print("\r Completion ",comp*100,end='')
    user_df=get_all_tweets(user)
    user_df["user"]=user
    user_df=user_df.drop(columns="id")
    tweet_df=pd.concat([tweet_df, user_df],axis=0,sort=False)

 Completion  55.00000000000001

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

Let's save the extracted dataFrame to a csv file: 


In [ ]:
tweet_df.to_csv("users_0_55")

Let's load it:

In [103]:
tweet_df=pd.read_csv("users_0_55.csv",delimiter=',')
tweet_df.head()

user           created_at  \
0  0406Guno  2020-04-04 19:30:40   
1  0406Guno  2020-04-04 19:03:01   
2  0406Guno  2020-04-04 15:03:57   
3  0406Guno  2020-04-04 12:22:57   
4  0406Guno  2020-04-04 12:18:52   

                                                text  
0  b'#CoronavirusOutbreak : bodies are being left...  
1  b'couple names twins #Corona and #Covid at Chh...  
2  b'LIVE: NY Gov. #Cuomo provides latest #corona...  
3  b'RT @globaltimesnews: China stopped for 3 min...  
4  b'RT @UNICEF: Be kind.\n\n#COVID19 doesn\xe2\x...

Well it seems that I have been blocked by Twitter. Let's load another csv file after running the same on a second compute using 4G connection.

In [104]:
tweet_2_df=pd.read_csv("users_100_150.csv",delimiter=',')
tweet_2_df=tweet_2_df.drop(columns='index')
tweet_2_df.tail()

user           created_at  \
137006  Care2much18  2020-02-04 05:53:37   
137007  Care2much18  2020-02-04 05:47:15   
137008  Care2much18  2020-02-04 05:40:58   
137009  Care2much18  2020-02-04 05:40:46   
137010  Care2much18  2020-02-04 05:40:06   

                                                     text  
137006  b'~48% of them bigoted enough to tell people t...  
137007  b"209 million adult in the US and yet you're p...  
137008  b'RT @benfowlkesMMA: We sure do cram a lot of ...  
137009  b"RT @lthomasnews: There seems to be confusion...  
137010  b'RT @humorandanimals: when you leave dad alon...

In [106]:
tweet_3_df=pd.read_csv("users_250_300.csv",delimiter=',')
tweet_3_df.tail()

user           created_at  \
157339  HomelandKen  2019-06-14 19:58:35   
157340  HomelandKen  2019-06-14 16:33:57   
157341  HomelandKen  2019-06-13 18:26:09   
157342  HomelandKen  2019-06-13 18:26:08   
157343  HomelandKen  2019-06-13 18:26:08   

                                                     text  
157339  b'This is a great example of why we need Congr...  
157340  b'Today, @USCIS took action to further @POTUS ...  
157341  b'I look forward to working with the men and w...  
157342  b'Our nation has the most generous legal immig...  
157343  b"I'm honored to be given the opportunity to l...

### Remove ponctuation

In [107]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

tweet_df['text']=tweet_df['text'].astype(str)
tweet_df['text'] = tweet_df['text'].apply(lambda x: remove_punct(x))

tweet_2_df['text']=tweet_2_df['text'].astype(str)
tweet_2_df['text'] = tweet_2_df['text'].apply(lambda x: remove_punct(x))

tweet_3_df['text']=tweet_3_df['text'].astype(str)
tweet_3_df['text'] = tweet_3_df['text'].apply(lambda x: remove_punct(x))

tweet_3_df.head(10)

user           created_at  \
0  FL_HealthPlans  2020-03-30 18:03:01   
1  FL_HealthPlans  2020-03-22 15:15:41   
2  FL_HealthPlans  2020-03-21 14:18:17   
3  FL_HealthPlans  2020-03-17 02:58:53   
4  FL_HealthPlans  2020-03-17 02:45:38   
5  FL_HealthPlans  2020-02-03 06:07:05   
6  FL_HealthPlans  2020-01-21 02:59:59   
7  FL_HealthPlans  2019-12-11 20:36:17   
8  FL_HealthPlans  2019-11-27 23:55:04   
9  FL_HealthPlans  2019-11-26 01:25:23   

                                                text  
0  bRT HealthyFla Stay updated with Floridas COVI...  
1  bPrayTogether xfxfxxf xfxfxcxexfxfxcxfxfxfxcxd...  
2  bStopTheSpread CoronavirusPandemic SocialDista...  
3  bRT GovRonDeSantis If you are experiencing the...  
4  bcoronavirus QuarantineLife DontBeASpreader He...  
5  bGodBlessAmerica xfxfxxbaxfxfxxb httpstcoFclhR...  
6  bNeverForget Peace Love Respect httpstcoTrTwFT...  
7  bShareALittleSun SpringsRUsFla Blue Hole Sprin...  
8  bLoveFL Sunset Florida is paradise httpstcofUh...  
9  bLoveFL Another beautiful sunset httpstcoenlqc...

### Tokenization

In [108]:
def tokenization(text):
    text = re.split('\W+', text)
    return text
tweet_df['text']=tweet_df['text'].str[1:]
tweet_df['text_tokenized'] = tweet_df['text'].apply(lambda x: tokenization(x.lower()))

tweet_2_df['text']=tweet_2_df['text'].str[1:]
tweet_2_df['text_tokenized'] = tweet_2_df['text'].apply(lambda x: tokenization(x.lower()))

tweet_3_df['text']=tweet_3_df['text'].str[1:]
tweet_3_df['text_tokenized'] = tweet_3_df['text'].apply(lambda x: tokenization(x.lower()))

tweet_3_df.tail()

user           created_at  \
157339  HomelandKen  2019-06-14 19:58:35   
157340  HomelandKen  2019-06-14 16:33:57   
157341  HomelandKen  2019-06-13 18:26:09   
157342  HomelandKen  2019-06-13 18:26:08   
157343  HomelandKen  2019-06-13 18:26:08   

                                                     text  \
157339  This is a great example of why we need Congres...   
157340  Today USCIS took action to further POTUS direc...   
157341  I look forward to working with the men and wom...   
157342  Our nation has the most generous legal immigra...   
157343  Im honored to be given the opportunity to lead...   

                                           text_tokenized  
157339  [this, is, a, great, example, of, why, we, nee...  
157340  [today, uscis, took, action, to, further, potu...  
157341  [i, look, forward, to, working, with, the, men...  
157342  [our, nation, has, the, most, generous, legal,...  
157343  [im, honored, to, be, given, the, opportunity,...

### Stopwords removal

In [109]:
stopword = nltk.corpus.stopwords.words('english')
stopword.extend(["rt","RT"])
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

In [110]:

tweet_df['text_tokenized'] = tweet_df['text_tokenized'].apply(lambda x: remove_stopwords(x))
tweet_2_df['text_tokenized'] = tweet_2_df['text_tokenized'].apply(lambda x: remove_stopwords(x))
tweet_3_df['text_tokenized'] = tweet_3_df['text_tokenized'].apply(lambda x: remove_stopwords(x))

tweet_3_df.tail(10)

user           created_at  \
157334  HomelandKen  2019-06-15 00:31:18   
157335  HomelandKen  2019-06-14 21:07:19   
157336  HomelandKen  2019-06-14 21:07:18   
157337  HomelandKen  2019-06-14 21:07:18   
157338  HomelandKen  2019-06-14 19:58:36   
157339  HomelandKen  2019-06-14 19:58:35   
157340  HomelandKen  2019-06-14 16:33:57   
157341  HomelandKen  2019-06-13 18:26:09   
157342  HomelandKen  2019-06-13 18:26:08   
157343  HomelandKen  2019-06-13 18:26:08   

                                                     text  \
157334  An affidavit of support is a legal and enforce...   
157335  At FlagDay naturalization ceremonies today leg...   
157336  Through assimilation legal immigrants show the...   
157337  On June  we honor one of the most cherished sy...   
157338  This Administration is committed to protecting...   
157339  This is a great example of why we need Congres...   
157340  Today USCIS took action to further POTUS direc...   
157341  I look forward to working with the men and wom...   
157342  Our nation has the most generous legal immigra...   
157343  Im honored to be given the opportunity to lead...   

                                           text_tokenized  
157334  [affidavit, support, legal, enforceable, contr...  
157335  [flagday, naturalization, ceremonies, today, l...  
157336  [assimilation, legal, immigrants, show, commit...  
157337  [june, honor, one, cherished, symbols, great, ...  
157338  [administration, committed, protecting, americ...  
157339  [great, example, need, congress, take, action,...  
157340  [today, uscis, took, action, potus, directive,...  
157341  [look, forward, working, men, women, uscis, en...  
157342  [nation, generous, legal, immigration, system,...  
157343  [im, honored, given, opportunity, lead, uscis,...

## Concatenation

In [111]:
tweet_all=pd.concat([tweet_df, tweet_2_df,tweet_3_df],axis=0,sort=False)
tweet_all=tweet_all.reset_index(drop=True)
tweet_all

user           created_at  \
0          0406Guno  2020-04-04 19:30:40   
1          0406Guno  2020-04-04 19:03:01   
2          0406Guno  2020-04-04 15:03:57   
3          0406Guno  2020-04-04 12:22:57   
4          0406Guno  2020-04-04 12:18:52   
...             ...                  ...   
446758  HomelandKen  2019-06-14 19:58:35   
446759  HomelandKen  2019-06-14 16:33:57   
446760  HomelandKen  2019-06-13 18:26:09   
446761  HomelandKen  2019-06-13 18:26:08   
446762  HomelandKen  2019-06-13 18:26:08   

                                                     text  \
0       CoronavirusOutbreak  bodies are being left on ...   
1       couple names twins Corona and Covid at Chhatti...   
2       LIVE NY Gov Cuomo provides latest coronavirus ...   
3       RT globaltimesnews China stopped for  minutes ...   
4       RT UNICEF Be kindnnCOVID doesnxexxt discrimina...   
...                                                   ...   
446758  This is a great example of why we need Congres...   
446759  Today USCIS took action to further POTUS direc...   
446760  I look forward to working with the men and wom...   
446761  Our nation has the most generous legal immigra...   
446762  Im honored to be given the opportunity to lead...   

                                           text_tokenized  
0       [coronavirusoutbreak, bodies, left, sidewalks,...  
1       [couple, names, twins, corona, covid, chhattis...  
2       [live, ny, gov, cuomo, provides, latest, coron...  
3       [globaltimesnews, china, stopped, minutes, sat...  
4       [unicef, kindnncovid, doesnxexxt, discriminate...  
...                                                   ...  
446758  [great, example, need, congress, take, action,...  
446759  [today, uscis, took, action, potus, directive,...  
446760  [look, forward, working, men, women, uscis, en...  
446761  [nation, generous, legal, immigration, system,...  
446762  [im, honored, given, opportunity, lead, uscis,...  

[446763 rows x 4 columns]

## Let's try to keep only tweets on Covid_19 outbreak

In [112]:
corona_keywords = ['corona', 'coronavirus','covid','virus','outbreak','disease'] 
contains_corona_keywords = lambda x: any([(kw in str(x.lower())) for kw in corona_keywords])

In [113]:
# Check for text matching the coronavirus keywords in the 4 columns
corona_tweets = (tweet_all['text'].apply(contains_corona_keywords)) \
             

tweet_all['corona'] = corona_tweets

print('There are {} corona tweets, and {} no corona  tweets.'.format(tweet_all['corona'].sum(), 
                                                len(tweet_all['corona']) - tweet_all['corona'].sum()))
corona_df=tweet_all[tweet_all['corona'] == True]
corona_df.head()

There are 55265 corona tweets, and 391498 no corona  tweets.


user           created_at  \
0  0406Guno  2020-04-04 19:30:40   
1  0406Guno  2020-04-04 19:03:01   
2  0406Guno  2020-04-04 15:03:57   
3  0406Guno  2020-04-04 12:22:57   
4  0406Guno  2020-04-04 12:18:52   

                                                text  \
0  CoronavirusOutbreak  bodies are being left on ...   
1  couple names twins Corona and Covid at Chhatti...   
2  LIVE NY Gov Cuomo provides latest coronavirus ...   
3  RT globaltimesnews China stopped for  minutes ...   
4  RT UNICEF Be kindnnCOVID doesnxexxt discrimina...   

                                      text_tokenized  corona  
0  [coronavirusoutbreak, bodies, left, sidewalks,...    True  
1  [couple, names, twins, corona, covid, chhattis...    True  
2  [live, ny, gov, cuomo, provides, latest, coron...    True  
3  [globaltimesnews, china, stopped, minutes, sat...    True  
4  [unicef, kindnncovid, doesnxexxt, discriminate...    True

In [114]:
corona_df.head(10)

user           created_at  \
0   0406Guno  2020-04-04 19:30:40   
1   0406Guno  2020-04-04 19:03:01   
2   0406Guno  2020-04-04 15:03:57   
3   0406Guno  2020-04-04 12:22:57   
4   0406Guno  2020-04-04 12:18:52   
6   0406Guno  2020-04-04 09:49:41   
9   0406Guno  2020-04-03 15:10:21   
11  0406Guno  2020-04-03 12:01:11   
15  0406Guno  2020-04-02 07:32:14   
17  0406Guno  2020-04-01 22:07:53   

                                                 text  \
0   CoronavirusOutbreak  bodies are being left on ...   
1   couple names twins Corona and Covid at Chhatti...   
2   LIVE NY Gov Cuomo provides latest coronavirus ...   
3   RT globaltimesnews China stopped for  minutes ...   
4   RT UNICEF Be kindnnCOVID doesnxexxt discrimina...   
6   US nurse quits after hospital says no wearing ...   
9   LIVE NY Gov Cuomo provides latest coronavirus ...   
11  Italian governor I was called racist for wanti...   
15  The world will see confirmed COVID cases reach...   
17  The WhiteHouse novel coronavirus task force gi...   

                                       text_tokenized  corona  
0   [coronavirusoutbreak, bodies, left, sidewalks,...    True  
1   [couple, names, twins, corona, covid, chhattis...    True  
2   [live, ny, gov, cuomo, provides, latest, coron...    True  
3   [globaltimesnews, china, stopped, minutes, sat...    True  
4   [unicef, kindnncovid, doesnxexxt, discriminate...    True  
6   [us, nurse, quits, hospital, says, wearing, ma...    True  
9   [live, ny, gov, cuomo, provides, latest, coron...    True  
11  [italian, governor, called, racist, wanting, t...    True  
15  [world, see, confirmed, covid, cases, reach, o...    True  
17  [whitehouse, novel, coronavirus, task, force, ...    True